In [1]:
library(dplyr)
library(MASS)    # for negative binomial regression
library(readr)   # read csv file


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Attaching package: 'MASS'

The following object is masked from 'package:dplyr':

    select



### <div class='info-circle alert alert-block alert-info'> Load combined data</div>

In [2]:
reg_df <- read_csv('Data/combined_data.csv')
dim(reg_df)

Parsed with column specification:
cols(
  .default = col_double(),
  city_cn = col_character(),
  city = col_character(),
  province_cn = col_character(),
  province = col_character(),
  tier = col_character()
)
See spec(...) for full column specifications.


[1] 366  97

In [3]:
head(reg_df)

city_cn,city,city_id,province_cn,province,tier,lat,lng,population,GDP,...,date_0324,date_0325,date_0326,date_0327,date_0328,date_0329,date_0330,date_0331,date_0401,hubei_city
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
北京市,Beijing,110000,北京市,Beijing,Tier 1,39.91092,116.4134,2153.60,35371.30,...,559,565,569,572,576,577,580,580,582,0
天津市,Tianjin,120000,天津市,Tianjin,new Tier 1,39.09367,117.2095,1561.83,14104.00,...,145,147,152,156,163,166,174,174,176,0
石家庄市,Shijiazhuang,130100,河北省,Hebei,Tier 2,38.04831,114.5215,1039.42,5809.90,...,29,29,29,29,29,29,29,29,29,0
唐山市,Tangshan,130200,河北省,Hebei,Tier 3,39.63658,118.1865,796.40,6890.00,...,58,58,58,58,58,58,58,58,58,0
秦皇岛市,Qinhuangdao,130300,河北省,Hebei,Tier 3,39.94175,119.6085,314.63,1612.02,...,10,10,10,10,10,10,10,10,10,0
邯郸市,Handan,130400,河北省,Hebei,Tier 3,36.63126,114.5456,954.97,3486.00,...,32,32,32,32,32,32,32,32,32,0


In [4]:
# Filter Wuhan
reg_df <-
reg_df %>% 
    filter(city != 'Wuhan')

### <div class="alert alert-block alert-danger">Outbreak Estimates</div>

* COVID-19 outbreak in other cities

#### Outbreak case Prediction using Negative Binomial Model

1. For the outbreak in Wuhan, a Binomial regression model is fitted first;
2. When the outbreak area changes, the corresponding predictors (e.g., human flow from and the distance to the outbreak city) also vary with the change;
3. Assuming that the spatial spread pattern of COVID-19 remains unchanged (which is a very strong assumption);
4. Based on the adjusted predictors, estimations are then made based on the fitted model.

Formula:


NB(cumulative_cases) ~ log_local_flow + log_population + log_distance + mean_intensity

1. local_flow: human flow from the outbreak city to each city (vary with the outbreak city)
2. population: the population of each city (unchanged)
3. distance: distance from each city to the outbreak city (vary with the outbreak city)
4. mean_intensity: mean intra-city activity intensity (unchanged)

---
#### step 1: model fit

In [5]:
# negative Binomial model
# in the MASS package

# cases on '2020-02-09'
reg_df$cumulative_cases <- reg_df$date_0209

reg_df <- reg_df %>% 
                mutate(log_local_flow = log(local_flow + 1)) %>%
                mutate(log_population = log(population)) %>%
                mutate(log_distance = log(distance + 1))


negb_model <- glm.nb(cumulative_cases ~ 
                    log_local_flow + 
                    log_population + 
                    log_distance + 
                    mean_intensity, 
              data = reg_df
                   )

summary(negb_model)


Call:
glm.nb(formula = cumulative_cases ~ log_local_flow + log_population + 
    log_distance + mean_intensity, data = reg_df, init.theta = 2.051967158, 
    link = log)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.7898  -0.9566  -0.3890   0.2487   4.4077  

Coefficients:
               Estimate Std. Error z value Pr(>|z|)    
(Intercept)     0.02088    0.83420   0.025   0.9800    
log_local_flow  0.79124    0.05517  14.341  < 2e-16 ***
log_population  0.32241    0.05741   5.616 1.95e-08 ***
log_distance   -0.16561    0.09794  -1.691   0.0908 .  
mean_intensity  0.07306    0.06863   1.065   0.2871    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for Negative Binomial(2.052) family taken to be 1)

    Null deviance: 2332.22  on 363  degrees of freedom
Residual deviance:  430.82  on 359  degrees of freedom
  (1 observation deleted due to missingness)
AIC: 2750.7

Number of Fisher Scoring iterations: 1


             

In [6]:
1/negb_model$theta

[1] 0.4873372

---
#### step 2: model prediciton when the outbreak city changes

##### case for Chengdu

In [7]:
# case example
city <- 'Chengdu'

nb_predict_base_df = read.csv(paste0('Data/outbreak_base_data/nb_base_', city, '_for_prediction.csv'))

head(nb_predict_base_df)

# 
# the human flow from Chengdu to Beijing is: 237.216012
# the distance from Beijing to Chengdu is: 1518.037

city,city_id,local_flow,population,distance,mean_intensity
<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
Beijing,110000,237.216012,2153.60,1518.037,5.677559
Tianjin,120000,40.574199,1561.83,1519.337,5.610923
Shijiazhuang,130100,36.633426,1039.42,1261.403,5.479823
Tangshan,130200,11.380362,796.40,1622.828,5.583632
Qinhuangdao,130300,5.695457,314.63,1743.796,5.384295
Handan,130400,20.725976,954.97,1174.229,5.281932


In [8]:
nb_predict_base_df <- nb_predict_base_df %>% 
                mutate(log_local_flow = log(local_flow + 1)) %>%
                mutate(log_population = log(population)) %>%
                mutate(log_distance = log(distance + 1))

In [9]:
predicted_cases <- predict(negb_model, nb_predict_base_df, type = "response")

# excluding NA: the outbreak city itsself and cities with missing values
raw_predicted = sum(predicted_cases, na.rm=TRUE)
round_predicted = sum(round(predicted_cases), na.rm=TRUE)

cat(city, ', ', raw_predicted, ', ', round_predicted, '\n')

Chengdu ,  30492.83 ,  30495 


##### multiple cities

In [10]:

cities <- c('Beijing', 'Shanghai', 'Tianjin', 'Chongqing', 
            'Zhengzhou', 'Changsha', 'Hefei', 'Guangzhou', 
            'Shenzhen', 'Nanchang', 'Nanjing', 'Hangzhou', 
            'Shenyang', 'Suzhou', 'Xian', 'Chengdu')

cities_cn <- c('北京市', '上海市', '天津市', '重庆市', 
              '郑州市', '长沙市', '合肥市', '广州市', 
              '深圳市', '南昌市', '南京市', '杭州市', 
              '沈阳市', '苏州市', '西安市', '成都市')

raw_predicts <- c()
round_predicts <- c()

for (city in cities)
{
    nb_predict_base_df = read.csv(paste0('Data/outbreak_base_data/nb_base_', 
                                   city, '_for_prediction.csv'))
    nb_predict_base_df <- nb_predict_base_df %>% 
                mutate(log_local_flow = log(local_flow + 1)) %>%
                mutate(log_population = log(population)) %>%
                mutate(log_distance = log(distance + 1))

    predicted_cases <- predict(negb_model, nb_predict_base_df, type = "response")
    raw_predicted = sum(predicted_cases, na.rm=TRUE)
    round_predicted = sum(round(predicted_cases), na.rm=TRUE)
    cat(city, ', ', raw_predicted, ', ', round_predicted, '\n')
    # cities <- rbind(cities, city)
    raw_predicts <- rbind(raw_predicts, raw_predicted)
    round_predicts <- rbind(round_predicts, round_predicted)
}

# cities_cn
# cities
# raw_predicts
# round_predicts

Beijing ,  43722.09 ,  43719 
Shanghai ,  40726.77 ,  40737 
Tianjin ,  20707.53 ,  20707 
Chongqing ,  18023.89 ,  18016 
Zhengzhou ,  26258.65 ,  26253 
Changsha ,  21162.38 ,  21168 
Hefei ,  16151.74 ,  16154 
Guangzhou ,  43819.59 ,  43824 
Shenzhen ,  40363.96 ,  40363 
Nanchang ,  11134.13 ,  11125 
Nanjing ,  21501.9 ,  21495 
Hangzhou ,  27439.6 ,  27432 
Shenyang ,  12126.04 ,  12131 
Suzhou ,  33250.31 ,  33251 
Xian ,  22668.72 ,  22664 
Chengdu ,  30492.83 ,  30495 


In [11]:
predicted <-
data.frame(city_cn=cities_cn, city=cities, 
           raw_predict=raw_predicts, predicted_cumulative_cases=round_predicts)

rownames(predicted) = 1:nrow(predicted)
predicted

city_cn,city,raw_predict,predicted_cumulative_cases
<fct>,<fct>,<dbl>,<dbl>
北京市,Beijing,43722.09,43719
上海市,Shanghai,40726.77,40737
天津市,Tianjin,20707.53,20707
重庆市,Chongqing,18023.89,18016
郑州市,Zhengzhou,26258.65,26253
长沙市,Changsha,21162.38,21168
合肥市,Hefei,16151.74,16154
广州市,Guangzhou,43819.59,43824
深圳市,Shenzhen,40363.96,40363


In [12]:
# results to file
file_path = file.path(paste0("Data/", "negb_predicted_outbreak_size_by_area.csv"))
# write.csv(predicted, file = file_path, row.names = FALSE, fileEncoding = "UTF-8")
